In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from random import random, seed
from math import exp

##### 1. Initialize Network

In [2]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create a list named 'network' in which I contain hidden and output layer
    network = []
    
    # Create a hidden layer
    hidden_layer = [{'weights' : [random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    # Create a output layer
    output_layer = [{'weights' : [random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

##### 2. Forward Propagate Inputs

##### 2-1. Neuron Activation

In [3]:
# Calculate neuron activaton for an input
def activate(weights, inputs):
    # Include the bias for calculation
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation    

##### 2-2. Neuron Transfer

In [4]:
# Transfer neuron activation
def transfer(activation):
    # Sigmoid(Logistic) function is applied
    return 1.0 / (1.0 + exp(-activation))

##### 2-3. Forward Propagate input to a network output

In [25]:
def forward_propagate(network, row):
    # Create a variable 'input' to forward propagate the neurons in the input layer to the hidden layer
    # and save the values in row to the variable 'input'.
    inputs = row
    
    # Loop the network list to access the layers sequentially
    for layer in network:
        # Create a variable 'new_inputs' to contain the values that result from activate and transfer.
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
            
        # We need this to be used as the input to the next layer
        inputs = new_inputs
        
    return inputs

##### 3. Back Propagate Error

##### 3-1. Transfer Derivative

In [6]:
def transfer_derivative(output):
    # Sigmoid(Logistic) funtion is used.
    return output * (1.0 - output)

##### 3-2. Error Backpropagation

In [7]:
def backward_propagate_error(network, expected):
    for i in range(len(network)-1, -1, -1):
        
        # Start with the output layer because outputs flows backwards from output layer to input layer.
        layer = network[i]   # output layer
        # error_signal = error * transfer_derivative(output) for the output layer
        errors, error_signals = [],[]  
        
        for k, neuron in enumerate(layer):
            if i == len(network) - 1:
                error = neuron['output'] - expected[k]
                error_signal = error * transfer_derivative(neuron['output'])
            else:
                error = 0.0
                for j, next_neuron in enumerate(network[i+1]):
                    error += next_neuron['weights'][k] * next_neuron['delta']
                    error_signal = error * transfer_derivative(neuron['output'])
                    
            errors.append(error)
            error_signals.append(error_signal)
            neuron['delta'] = error_signal

In [26]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


##### 4. Train Network

##### 4-1. Update network weights

In [27]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i, layer in enumerate(network):
        if i == 0:
            inputs = row[:-1]     # last element is the target variable
        else:
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in layer:
            for k, input in enumerate(inputs):
                neuron['weights'][k] -= l_rate * neuron['delta'] * input
                neuron['weights'][-1] -- l_rate * neuron['delta']

##### Train Network

In [46]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    # Implement Stochastic Gradient Descent
    for epoch in range(n_epoch):
        sum_error = 0
        # For each epoch, train the network by minimizing sum_error
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1 
            
            sum_error += sum([(expected[i] - outputs[i]) ** 2 for i in range(len(expected))])
        backward_propagate_error(network, expected)
        update_weights(network, row, l_rate) 
        # print(">epoch=%d, lrate=%.3f, error=%.3f"% (epoch, l_rate, sum_error))
        print(f"epoch : {epoch>2d}, lrate : {l_rate}, error : {sum_error}")    

SyntaxError: f-string: invalid syntax (2521316650.py, line 16)

In [42]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
 [1.465489372,2.362125076,0],
 [3.396561688,4.400293529,0],
 [1.38807019,1.850220317,0],
 [3.06407232,3.005305973,0],
 [7.627531214,2.759262235,1],
 [5.332441248,2.088626775,1],
 [6.922596716,1.77106367,1],
 [8.675418651,-0.242068655,1],
 [7.673756466,3.508563011,1]]

In [43]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)

for layer in network:
    print(layer)
    print()

>epoch=0, lrate=0.500, error=6.713
>epoch=1, lrate=0.500, error=6.635
>epoch=2, lrate=0.500, error=6.550
>epoch=3, lrate=0.500, error=6.460
>epoch=4, lrate=0.500, error=6.367
>epoch=5, lrate=0.500, error=6.277
>epoch=6, lrate=0.500, error=6.193
>epoch=7, lrate=0.500, error=6.121
>epoch=8, lrate=0.500, error=6.066
>epoch=9, lrate=0.500, error=6.030
>epoch=10, lrate=0.500, error=6.015
>epoch=11, lrate=0.500, error=6.020
>epoch=12, lrate=0.500, error=6.043
>epoch=13, lrate=0.500, error=6.080
>epoch=14, lrate=0.500, error=6.129
>epoch=15, lrate=0.500, error=6.186
>epoch=16, lrate=0.500, error=6.248
>epoch=17, lrate=0.500, error=6.313
>epoch=18, lrate=0.500, error=6.380
>epoch=19, lrate=0.500, error=6.447
[{'weights': [0.13007336290078605, 0.8454718780783583, 0.763774618976614], 'output': 0.9911266714210061, 'delta': -0.00035184083782383035}, {'weights': [0.25192063083482386, 0.49399559106998153, 0.4494910647887381], 'output': 0.983541865395868, 'delta': -0.0007175332832771064}]

[{'weights

In [30]:
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)

for layer in network:
    print(layer)

[1, 0]
[0.8094918973879515, 0.7734292563511262]


TypeError: 'float' object is not subscriptable

In [12]:
for i in range(len(expected)):
    expected = [0 for i in range(3)]
    expected[i] = 1
    print(expected)

[1, 0, 0]
[0, 1, 0]
[0, 0, 1]


In [17]:
outputs = [1,0,2,0,1,5]
for output in outputs:
    expected = [0 for j in range(len(outputs))]
    expected[output] = 1
    print(expected)

[0, 1, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0]
[1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1]
